# hennelly_ml.ipynb  
**Author:** Kevin Hennelly  

[GitHub Repository](https://github.com/KHenn22/datafun-07-ml)

In [ ]:
# Import statements and setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Optional: setup or naming conventions
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')